This Notebook contains the Python code written for the paper ''Fiscalflation: Fiscal implications for inflation in post-pandemic Germany'' by Florian Frühhaber. It also produces additional graphs used in the working paper version.

- [Figure 1](#fig1)
- [Figure 2a](#fig2a)
- [Figure 2b](#fig2b)
- [Figure 3](#fig3)
- [Figure 4](#fig4)
- [Figure 5](#fig5)
- [Figure 6a](#fig6a)
- [Figure 6b](#fig6b)
- [Figure 7](#fig7)

In [1]:
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import warnings

#warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
#warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

# from IPython.display import display, HTML
# display(HTML("<style>.container { width:60% !important; }</style>"))
# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## Data generation

1. Get data about inflation expectations from ECB's SPF and combine it with own assumptions (Inflation expectations constant after 5 years, 2% after 15 year)

In [2]:
# url = 'https://www.ecb.europa.eu/stats/ecb_surveys/survey_of_professional_forecasters/html/table_hist_hicp.en.html'
# html = requests.get(url).content
# df_list = pd.read_html(html)
# df = df_list[-1]

# data = df.rename(columns={'One year ahead':'pi1','Two years ahead':'pi2',df.columns[6]:'pi5'}).drop(columns=['Current calendar year','Next calendar year','Calendar yearafter next'])
# data['pi3'] = round((data['pi5']-data['pi2'])/3 + data['pi2'],1)
# data['pi4'] = round((data['pi5']-data['pi2'])*2/3 + data['pi2'],1)
# for i in range(6,16):
#     data['pi'+str(i)] = data['pi5']
# for i in range(16,31):
#     data['pi'+str(i)] = [2]*len(data['pi5'])
# data

# #

# data2 = data.transpose()
# data2.columns = data2.iloc[0]
# data2 = data2.drop(index="Survey round")
# for y in range(1999,2024):
#         for m in range(1,13):
#             try:
#                 if m in [1,2,3]:
#                     data2[str(m)+"-"+str(y)] = data2[str(y)+" Q1"]  
#                 elif m in [4,5,6]:
#                     data2[str(m)+"-"+str(y)] = data2[str(y)+" Q2"]
#                 elif m in [7,8,9]:
#                     data2[str(m)+"-"+str(y)] = data2[str(y)+" Q3"]
#                 else:
#                     data2[str(m)+"-"+str(y)] = data2[str(y)+" Q4"]
#             except:
#                 break

# data2.to_excel('spf_inflation_expectations.xlsx')
# This output is in sheet 'SPF Inflation' in data.xlsx

2. [In Stata] Use ECB nominal Yield curve to calculate implied short-term interest rate expectations (see 'Yield Curve.do') --> output copied into 'data.xlsx/EA E(i) from YC'

3. [In Excel] Calculate expected short-term real interest rates by substracting inflation expectations from nominal interest rates (see sheet 'E(r) SPF + YC')

4. Manually collect German public data on debt from various sources (as linked)

In [3]:
years = [2019,2020,2021,2022]

# Net debt as in WEO (Oct 21 - Apr 21 - Apr 23 - Apr 23)
# https://www.imf.org/en/Publications/WEO/weo-database/2021/October
net_debt_weo = [1418.23, 1558.63, 1641.67, 1742.99]
net_debt_weo_rev = [1391.21, 1545.25, 1641.67, 1742.99] # all from Apr 23

# Gross debt as in WEO (Oct 21 - Oct 21 - Apr 23 - Apr 23))
# https://www.imf.org/en/Publications/WEO/weo-database/2021/October
gross_debt_weo = [2057.63, 2314.09, 2471.63, 2572.95]

# Gross debt (Eurostat)
# https://ec.europa.eu/eurostat/databrowser/explore/all/economy?lang=en&subtheme=gov.gov_gfs10.gov_10dd.gov_10dd_sgd&display=list&sort=category
gross_debt_eurostat = [2068.81, 2339.93, 2494.59, 2563.08]
gross_debt_eurostat_bund = [1312.68, 1527.06, 1679.84, 1776.88] # central government
gross_debt_eurostat_laender = [a-b for a,b in zip(gross_debt_eurostat,gross_debt_eurostat_bund)] # federal states

# Markt value of debt (Eurostat)
# https://ec.europa.eu/eurostat/databrowser/explore/all/economy?lang=en&subtheme=gov.gov_gfs10.gov_10dd.gov_10dd_sgd&display=list&sort=category
mv_debt_eurostat = [2273.31, 2661.92, 2742.53, 2391.29]
mv_debt_eurostat_bund = [1498.79, 1821.41, 1914.90, 1662.41]
mv_debt_eurostat_laender = [a-b for a,b in zip(mv_debt_eurostat, mv_debt_eurostat_bund)]

# Outstanding debt (Destatis)
# https://www-genesis.destatis.de/genesis//online?operation=table&code=71321-0007&bypass=true&levelindex=1&levelid=1687375607611#abreadcrumb
debt_destatis_bund = [1188.58, 1403.48, 1548.47, 1620.36]
debt_destatis_laender = [579.01, 635.97, 638.55, 606.83]
debt_destatis = [a+b for a,b in zip(debt_destatis_bund[:-1],debt_destatis_laender[:-1])] + [None]
#  -> used for analysis:
debt_bund_2019 = 1188.6
debt_laender_2019 = 580



net_debt_change = [None]
gross_debt_change = [None]
mv_debt_change = [None]
revaluation = [None]
for i in [1,2,3]:
    net_debt_change.append(net_debt_weo_rev[i] - net_debt_weo_rev[i-1])
    gross_debt_change.append(gross_debt_eurostat[i] - gross_debt_eurostat[i-1])
    mv_debt_change.append(mv_debt_eurostat[i] - mv_debt_eurostat[i-1])
    revaluation.append((mv_debt_eurostat[i] - mv_debt_eurostat[i-1])-(gross_debt_eurostat[i] - gross_debt_eurostat[i-1]))
    print('Changes in '+str(years[i])+':')
    print('        Net debt: '+str(net_debt_weo_rev[i] - net_debt_weo_rev[i-1]))
    print('        Gross debt: '+str(gross_debt_eurostat[i] - gross_debt_eurostat[i-1]))
    print('        Market Value of debt: '+str(mv_debt_eurostat[i] - mv_debt_eurostat[i-1]))
    print('        Valuation effect: '+str((mv_debt_eurostat[i] - mv_debt_eurostat[i-1])-(gross_debt_eurostat[i] - gross_debt_eurostat[i-1])))
    print('')
print('Total Valuation Effect over 2020-2021: '+str((mv_debt_eurostat[2] - mv_debt_eurostat[0])-
                                                        (gross_debt_eurostat[2] - gross_debt_eurostat[0])))
                          
df_debt = pd.DataFrame({'Year':years, 'Net debt':net_debt_weo_rev, 'Gross debt (WEO)':gross_debt_weo,
                  'Gross debt':gross_debt_eurostat, 'Gross debt (Eurostat) - Federal':gross_debt_eurostat_bund,
                  'Gross debt (Eurostat) - States':gross_debt_eurostat_laender,'Market value':mv_debt_eurostat,
                  'Market value (Eurostat) - Federal':mv_debt_eurostat_bund,'Market value (Eurostat) - States':mv_debt_eurostat_laender,
                  'Outstanding debt (Destatis)':debt_destatis,'Outstanding debt (Destatis) - Federal':debt_destatis_bund,
                  'Outstanding debt (Destatis) - States':debt_destatis_laender,'\u0394 net debt':net_debt_change,
                  '\u0394 gross debt':gross_debt_change,'\u0394 market value':mv_debt_change,
                  'Revaluation effect':revaluation})

Changes in 2020:
        Net debt: 154.03999999999996
        Gross debt: 271.1199999999999
        Market Value of debt: 388.6100000000001
        Valuation effect: 117.49000000000024

Changes in 2021:
        Net debt: 96.42000000000007
        Gross debt: 154.6600000000003
        Market Value of debt: 80.61000000000013
        Valuation effect: -74.05000000000018

Changes in 2022:
        Net debt: 101.31999999999994
        Gross debt: 68.48999999999978
        Market Value of debt: -351.24000000000024
        Valuation effect: -419.73

Total Valuation Effect over 2020-2021: 43.440000000000055


## Calculations and Figures

In [8]:
fig = px.bar(df_debt[df_debt['Year']>2019], x='Year',y=['\u0394 net debt','\u0394 gross debt','\u0394 market value'],
              color_discrete_sequence=["#E3E6E8","#CED4DA","#ADB5BD","#6C757D","#343A40","#131517"],
            barmode='group')

fig.update_layout(template="simple_white",
                 xaxis=dict(showgrid=False,dtick=1, tickfont = dict(size=16,family="serif")), 
                 yaxis=dict(showgrid=True,tickfont = dict(size=16,family="serif")))
#colors = ["#E3E6E8","#CED4DA","#ADB5BD","#6C757D","#343A40","#131517"]  

fig.add_trace(go.Scatter(
            x=df_debt[df_debt['Year']>2019]['Year'],
            y=df_debt[df_debt['Year']>2019]['Revaluation effect'],
            mode='markers',
            name='Revaluation effect',
            marker=dict(symbol='circle',size=9.5,color='black')
    ))

fig.update_layout(
    #title=f"<b>{title}</b>",
    yaxis_title="billion EUR",
    font_family="serif",
    font_size=14,
    xaxis_title=None,
    legend=dict(
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5,
        orientation="h",
        #bordercolor="black",
        #borderwidth=1,
        title="",
        font=dict(size=16,family="serif")
    ))

fig.write_image("debt measures change.pdf",width=800)
fig.show()

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


Calculation of numbers as described in section 3.2.5

In [ ]:
def interest_series(start,end,time):
    start=start
    end=end
    return [i/time*(end-start)+start for i in list(range(0,time))]+[end]*(32-time)

In [ ]:
def nominal_fiscal_balances(output='bund',baseyear=2020,ex_post=0,date="1-2020",
                                # Baseyear 2020: What is the situation like on the 01.01.2020?
                            growth_deviation=1, deviation_interest_rate=0, # for sensitivity checks
                            emergency_credit_states=[0,0,0], emergency_credit_bund=[0,0,0], # for years 2020, 2021, 2022
                            previous_debt_states=579.01, previous_debt_bund=1188.58, # initial values for 2020
                            interest_rate="", repayment_start=20, repayment_period=10,
                            rule_bund=0.0035,rule_states=0):

    if interest_rate == "":
        i_expected = list(pd.read_excel("data.xlsx", sheet_name='EA E(i) from YC')[date])
        interest_rate_bund = interest_series(1,1+i_expected[16]+deviation_interest_rate,15)[:18]+[1+i+deviation_interest_rate for i in i_expected[16:]]
        interest_rate_states = interest_series(1,1+i_expected[16]+deviation_interest_rate,15)[:18]+[1+i+deviation_interest_rate for i in i_expected[16:]]
    else:
        interest_rate_bund = interest_rate
        interest_rate_states = interest_rate        
    
    gdp = list(pd.read_excel("data.xlsx", sheet_name='gdp + output gap')[str(baseyear+ex_post)+' gdp'][:-2])
    if growth_deviation != 1:
        gdp = list(pd.read_excel("data.xlsx", sheet_name='gdp + output gap')[str(baseyear+ex_post)+' gdp'][:-2])[0:5]
        for i in range(1,28):
            gdp.append(gdp[4]*((1.03+growth_deviation)**i))
    
    df = pd.DataFrame({'Year':list(range(2020,2052)),
                 'GDP':gdp, 
                 'GDP gap':list(pd.read_excel("data.xlsx", sheet_name='gdp + output gap')[str(baseyear+ex_post)+' output gap'][:-2]),
                 'Debt Bund':[previous_debt_bund]+[None]*31,
                 'Debt States':[previous_debt_states]+[None]*31,
                 'Interest Rate Bund':[i for i in interest_rate_bund],
                 'Interest Rate States':[i for i in interest_rate_states]})
    
    emergency_credit_bund_sum = sum(emergency_credit_bund)
    emergency_credit_states_sum = sum(emergency_credit_states)
        
    df['NKA Bund'] = rule_bund*df['GDP'] - 0.203*df['GDP gap']
    df['NKA States'] = - rule_states*df['GDP gap'] - 0.301*df['GDP gap']
    
    # add borrowing and repayment Länder (fixed) --> emergency borrowing from different years is repaid together
    for i in range(0,3):
        df['NKA States'].iloc[i] = df['NKA States'].iloc[i] + emergency_credit_states[i]
    for i in range(0,10):
        df['NKA States'].iloc[i+11] = df['NKA States'].iloc[i+11] - (emergency_credit_states_sum/10)
    
    # add borrowing and repayment Bund
    for i in range(0,3):
        df['NKA Bund'].iloc[i] = df['NKA Bund'].iloc[i] + emergency_credit_bund[i]
    df['Tilgung Bund'] = [0]*repayment_start + [emergency_credit_bund_sum/repayment_period]*repayment_period + [0]*(32-repayment_period-repayment_start)
    df['NKA Bund'] -= df['Tilgung Bund']
    
    for i in range(0,32):
        df['Debt Bund'].loc[i+1] = df['Debt Bund'].loc[i] + df['NKA Bund'].loc[i]
        df['Debt States'].loc[i+1] = df['Debt States'].loc[i] + df['NKA States'].loc[i]
    df['Debt Total'] = df['Debt Bund'] + df['Debt States']

    df['Interest Bund'] = df['Interest Rate Bund']/100 * df['Debt Bund']
    df['Interest States'] = df['Interest Rate States']/100 * df['Debt States']

    df['Primary Surplus Bund'] = - df['NKA Bund'] + df['Interest Bund']
    df['Primary Surplus States'] = - df['NKA States'] + df['Interest States']
    df['Primary Surplus Total'] = - df['NKA Bund'] - df['NKA States'] + df['Interest Bund'] + df['Interest States']
    
    df['Primary Surplus Total'].iloc[-1] += 1630 # Scale Price Level in 2021 to 1
        # This step matters because when inflation expectations or discount rates change,
        # it also alters the PV of surpluses after 2050
    
    df['Interest Payments Total'] = (df['Interest Bund'] + df['Interest States']) / df['GDP']
    
    if output=='bund':
        return df['Primary Surplus Bund']
    elif output=='states':
        return df['Primary Surplus States']
    elif output=='total':
        return df['Primary Surplus Total']
    elif output=='dataframe':
        return df
    elif output=='Interest':
        return df['Interest Payments Total']
    else:
        print('Choose different output')

In [ ]:
def partial_product(series, n):
    product = 1
    for index, value in enumerate(series):
        if index < n:
            product *= value
    return product

In [ ]:
def expected_price_levels(base,deviation=0):
    expected_inflation = [i+deviation for i in list(pd.read_excel("data.xlsx", sheet_name='SPF Inflation')[base])]
    df = pd.DataFrame({'Expected Price Level': [1]*32})
    for i in range(1,32):
        df['Expected Price Level'].loc[i] = partial_product([j/100+1 for j in expected_inflation+[expected_inflation[-1]]*10],i)
    if base[-4:] == '2020':
        return list(df['Expected Price Level'])
    if int(base[-4:]) > 2020:
        liste = [1]*(int(base[-4:])-2020)+list(df['Expected Price Level'].iloc[:-(int(base[-4:])-2020)])
        return liste

In [ ]:
def real_discount_factors(base, deviation=0):
    discount = [i-deviation for i in list(pd.read_excel("data.xlsx", sheet_name='E(r) SPF + YC')[base])]
    df = pd.DataFrame({'Cumulated Discount Factor': [1]*32})
    for i in range(1,32):
        df['Cumulated Discount Factor'].loc[i] = 1/partial_product([j/100+1 for j in discount+[discount[-1]]*10],i)
    if base[-4:] == '2020':
        return list(df['Cumulated Discount Factor'])
    if int(base[-4:]) > 2020:
        liste = [1]*(int(base[-4:])-2020)+list(df['Cumulated Discount Factor'].iloc[:-(int(base[-4:])-2020)])
        return liste

In [ ]:
def discount_factors(base, deviation=0):
    discount = [i-deviation for i in list(pd.read_excel("data.xlsx", sheet_name='EA E(i) from YC')[base])]
    df = pd.DataFrame({'Cumulated Discount Factor': [1]*32})
    for i in range(1,32):
        df['Cumulated Discount Factor'].loc[i] = 1/partial_product([j/100+1 for j in discount+[discount[-1]]*10],i)
    if base[-4:] == '2020':
        return list(df['Cumulated Discount Factor'])
    if int(base[-4:]) > 2020:
        liste = [1]*(int(base[-4:])-2020)+list(df['Cumulated Discount Factor'].iloc[:-(int(base[-4:])-2020)])
        return liste

If one wants to add market valuations to the price level estimation in 2022, it would be necessary to apply this both to outstanding debt in 2019 and the additional debt from 2020-2021. In a first approximation, I only apply it to outstanding debt in 2019. 

| Year | Change market value | Change gross debt |
| ---- | ------------------- | ----------------- |
| 2020 | +17%                | +13%              |
| 2021 | +3%                 | +7%               |
| 2022 | \-13%               | +3%               |

In [ ]:
def price_level_estimation(fiscal_balances, discount_factors, debt = 1889.06,
                           year=0, output='price'):
    
    initial_debt=debt
    if year == 2020: 
        initial_debt = 1889.06*1.04
    elif year == 2021:
        initital_debt = 1889.06*1.04*1.04
    elif year == 2022:
        initital_debt = 1889.06*1.04*1.04*0.9
    
    df = pd.DataFrame({'Nominal Surplus':list(fiscal_balances), #fiscal_balances[(baseyear-2020)
                       'Discount Factor':discount_factors})
    df['Real Present Value of Surplus'] = df['Nominal Surplus'] * df['Discount Factor']
    total_present_value = sum(df['Real Present Value of Surplus'])
    price_level = initial_debt / total_present_value
    if output == 'price':
        return price_level
    elif output == 'dataframe':
        return df
    else:
        print('Choose different output')

Ex-post balances: https://ec.europa.eu/eurostat/databrowser/view/GOV_10DD_EDPT1__custom_7059583/default/table?lang=en

In [ ]:
# Counterfactuals for different budget laws on the federal level

# 2020

p2020_0 = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2020, date="1-2020")['Primary Surplus Total'],
        discount_factors = discount_factors("1-2020"))
    # Initial budget

df2020_1 = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="4-2020", # use 2021 bc GDP drop was already to be expected
                                    emergency_credit_bund = [99.8,0,0],
                                    emergency_credit_states = [30,0,0])
p2020_1a = price_level_estimation(
        fiscal_balances = df2020_1['Primary Surplus Total'],
        discount_factors = discount_factors("4-2020"))
    # S I    
p2020_1b = price_level_estimation(
        fiscal_balances = df2020_1['Primary Surplus Total'],
        discount_factors = discount_factors("1-2020"))
    # S I (constant discounting)
p2020_1c = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="4-2020",
                                    emergency_credit_bund = [0,0,0],
                                    emergency_credit_states = [0,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("4-2020"))
    # S I (no borrowing)
p2020_1d = price_level_estimation(year=2020,
        fiscal_balances = df2020_1['Primary Surplus Total'],
        discount_factors = discount_factors("4-2020"))
    # S I (with revaluation)   
p2020_1e = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2020", # use 2021 bc GDP drop was already to be expected
                                    emergency_credit_bund = [99.8,0,0],
                                    emergency_credit_states = [30,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("4-2020"))
    # S I (constant interest rates)       
    
df2020_2 = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="8-2020",
                                    emergency_credit_bund = [118.7,0,0],
                                    emergency_credit_states = [30,0,0])
p2020_2a = price_level_estimation(
        fiscal_balances = df2020_2['Primary Surplus Total'],
        discount_factors = discount_factors("8-2020"))
    # S II
p2020_2b = price_level_estimation(
        fiscal_balances = df2020_2['Primary Surplus Total'],
        discount_factors = discount_factors("1-2020"))
    # S II (constant discounting)
p2020_2c = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="8-2020",
                                    emergency_credit_bund = [0,0,0],
                                    emergency_credit_states = [0,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("8-2020"))
    # S II (no borrowing)
p2020_2d = price_level_estimation(year=2020,
        fiscal_balances = df2020_2['Primary Surplus Total'],
        discount_factors = discount_factors("8-2020"))
    # S II (with revaluation)
p2020_2e = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2020",
                                    emergency_credit_bund = [118.7,0,0],
                                    emergency_credit_states = [30,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("8-2020"))
    # S II (constant interest rates)

In [ ]:
# 2021

df2021_1 = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2021",
                                    emergency_credit_bund = [118.7,164.5,0],
                                    emergency_credit_states = [30,0,0])
p2021_1a = price_level_estimation(
        fiscal_balances = df2021_1['Primary Surplus Total'], # Include all state aid
        discount_factors = discount_factors("1-2021"))
    # Initial budget -> it was probably already clear that scope in 2020 would not be needed 
p2021_1b = price_level_estimation(
        fiscal_balances = df2021_1['Primary Surplus Total'], # Include all state aid
        discount_factors = discount_factors("1-2020"))
    # Initital budget (constant discounting)
p2021_1c = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2021",
                                    emergency_credit_bund = [0,0,0],
                                    emergency_credit_states = [0,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("1-2021"))
    # Initial budget (no borrowing)
p2021_1d = price_level_estimation(year=2020,
        fiscal_balances = df2021_1['Primary Surplus Total'], # Include all state aid
        discount_factors = discount_factors("1-2021"))
    # Initial budget (with revaluation) 
p2021_1e = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2020",
                                    emergency_credit_bund = [118.7,164.5,0],
                                    emergency_credit_states = [30,0,0])['Primary Surplus Total'], # Include all state aid
        discount_factors = discount_factors("1-2021"))
    # Initial budget (constant interest rates)

df2021_2 = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2021",
                                    emergency_credit_bund = [(118.7+41.9)/2,213.3,0],
                                    emergency_credit_states = [30,0,0])
p2021_2a = price_level_estimation(
        fiscal_balances = df2021_2['Primary Surplus Total'], 
        discount_factors = discount_factors("5-2021"))
    # S I
p2021_2b = price_level_estimation(
        fiscal_balances = df2021_2['Primary Surplus Total'],
        discount_factors = discount_factors("1-2020"))
    # S I (constant discounting)
p2021_2c = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="5-2021",
                                    emergency_credit_bund = [0,0,0],
                                    emergency_credit_states = [0,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("5-2021"))
    # S I (no borrowing)
p2021_2d = price_level_estimation(year=2021,
        fiscal_balances = df2021_2['Primary Surplus Total'], 
        discount_factors = discount_factors("5-2021"))
    # S I (with revaluation)
p2021_2e = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2020",
                                    emergency_credit_bund = [(118.7+41.9)/2,213.3,0],
                                    emergency_credit_states = [30,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("5-2021"))
    # S I (constant interest rates)


df2022 = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="10-2022",
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])
p2022a = price_level_estimation(
        fiscal_balances = df2022['Primary Surplus Total'],
        discount_factors = discount_factors("10-2022"))
    # ex post 2021
p2022b = price_level_estimation(
        fiscal_balances = df2022['Primary Surplus Total'],
        discount_factors = discount_factors("1-2020"))
    # ex post 2021 (constant discounting)
    
p2022c = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="10-2022",
                                    emergency_credit_bund = [0,0,0],
                                    emergency_credit_states = [0,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("10-2022"))
    # ex post 2021 (no borrowing)
p2022d = price_level_estimation(year=2022,
        fiscal_balances = df2022['Primary Surplus Total'],
        discount_factors = discount_factors("10-2022"))
    # ex post 2021 (with revaluation)
p2022e = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2020",
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("10-2022"))
    # ex post 2021 (constant interest rates)

df2022_2 = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2022",
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])
p2022_2a = price_level_estimation(year=2021,
        fiscal_balances = df2022_2['Primary Surplus Total'],
        discount_factors = discount_factors("1-2022"))
    # ex post at 1-2021
p2022_2b = price_level_estimation(
        fiscal_balances = df2022_2['Primary Surplus Total'],
        discount_factors = discount_factors("1-2020"))
    # ex post at 1-2021 (constant discounting)
    
p2022_2c = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2022",
                                    emergency_credit_bund = [0,0,0],
                                    emergency_credit_states = [0,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("1-2022"))
    # ex post at 1-2021 (no borrowing)
p2022_2d = price_level_estimation(year=2021,
        fiscal_balances = df2022_2['Primary Surplus Total'],
        discount_factors = discount_factors("1-2022"))
    # ex post at 1-2021 (with revaluation)
p2022_2e = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2020",
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("1-2022"))
    # ex post at 1-2021 (constant interest rates)

## Figure 2a
<a id='fig2a'></a>

with historic ex-post interest on total debt from Ameco

In [ ]:
implicit_interest = pd.read_excel("data.xlsx", sheet_name='Implicit interest rate')
implicit_interest = implicit_interest[implicit_interest['Year']<2021]
implicit_interest = pd.merge(implicit_interest,nominal_fiscal_balances('dataframe',date="1-2020")[['Year','Interest Rate Bund','Interest Rate States']],how='outer')

fig = px.line(implicit_interest, x='Year', y='Interest % of debt', color_discrete_sequence=[None])

fig.add_trace(go.Scatter(
    x=implicit_interest['Year'],
    y=implicit_interest['Interest % of debt'],
    mode='lines',
    name='Historic',
    connectgaps=True,
    line=dict(color='black')
))

fig.add_trace(go.Scatter(
    x=implicit_interest['Year'],
    y=implicit_interest['Interest Rate Bund']/100,
    mode='lines',
    name='1-2020',
    connectgaps=True,
    line=dict(color='#495057',dash="dashdot")
))

fig.add_trace(go.Scatter(
    x=df2021_1['Year'],
    y=df2021_1['Interest Rate Bund']/100,
    mode='lines',
    name='1-2021',
    marker=dict(symbol="square-open"),
    connectgaps=True,
    line=dict(color='#ADB5BD',dash="dot")
))


fig.add_trace(go.Scatter(
    x=df2022_2['Year'],
    y=df2022_2['Interest Rate Bund']/100,
    mode='lines',
    marker=dict(symbol="diamond-open"),
    name='1-2022',
    connectgaps=True,
    line=dict(color='#6C757D',dash="dash")
))

fig.add_trace(go.Scatter(
    x=df2022['Year'],
    y=df2022['Interest Rate Bund']/100,
    mode='lines',
    name='10-2022',
    marker=dict(symbol="diamond"),
    connectgaps=True,
    line=dict(color='#343A40',dash="dash")
))
    
    

fig.update_layout(template="simple_white",
                 xaxis=dict(showgrid=False,tickfont = dict(size=17,family="serif"),title=""), 
                 yaxis=dict(showgrid=True,tickfont = dict(size=17,family="serif"),tickformat="%",
                            title="Interest expenditures in % of total debt",titlefont = dict(size=18,family="serif")))
    
fig.update_layout(
    yaxis=dict(
    tickformat='.1%'
    ),
    width=700,
    legend=dict(
        yanchor="bottom",
        title="",
        y=1,
        xanchor="center",
        x=0.5,
        orientation="h",
        font=dict(size=18,family="serif")
    ))

fig.write_image('fig2a.pdf')
fig.show()

## Figure 4
<a id='fig4'></a>

In [ ]:
laws=['S I 2020 (3/2020)', 'S II 2020 (7/2020)', 'Initial 2021 (12/2020)', 'S I 2021 (4/2021)', 'Ex post (1-2022)', 'Ex post (9/2022)']
results_a = [p2020_1a/p2020_0-1, p2020_2a/p2020_0-1, p2021_1a/p2020_0-1, p2021_2a/p2020_0-1, p2022_2a/p2020_0-1, p2022a/p2020_0-1]
results_b = [p2020_1b/p2020_0-1, p2020_2b/p2020_0-1, p2021_1b/p2020_0-1, p2021_2b/p2020_0-1, p2022_2b/p2020_0-1, p2022b/p2020_0-1]
results_c = [p2020_1c/p2020_0-1, p2020_2c/p2020_0-1, p2021_1c/p2020_0-1, p2021_2c/p2020_0-1, p2022_2c/p2020_0-1, p2022c/p2020_0-1]
results_d = [p2020_1d/p2020_0-1, p2020_2d/p2020_0-1, p2021_1d/p2020_0-1, p2021_2d/p2020_0-1, p2022_2d/p2020_0-1, p2022d/p2020_0-1]
results_e = [p2020_1e/p2020_0-1, p2020_2e/p2020_0-1, p2021_1e/p2020_0-1, p2021_2e/p2020_0-1, p2022_2e/p2020_0-1, p2022e/p2020_0-1]

In [ ]:
fig = go.Figure()
fig.update_layout(template="simple_white",
                 xaxis=dict(showgrid=False,tickfont = dict(size=16,family="serif"),
                            title="Federal Budget Laws (Date)",titlefont = dict(size=18,family="serif")), 
                 yaxis=dict(showgrid=True,tickfont = dict(size=16,family="serif"),tickformat="0%",
                            title="PLE relative to 1-2020",titlefont = dict(size=18,family="serif")))
msize=8
fig.add_trace(go.Scatter(
    x=laws,
    y=results_a,
    mode='markers+text',
    name='Price Level Effect (PLE)',
    marker={'size':msize+2,'symbol':'square'},
    connectgaps=True,
    text=[round(i*100,1) for i in results_a],
    textposition="middle right",  # Positioning text to the right of the marker
    textfont=dict(color="#343A40"),
    line=dict(color='#343A40')
))


fig.add_trace(go.Scatter(
    x=laws,
    y=results_b,
    mode='markers',
    name='with constant discounting',
    marker={'size':msize,'symbol':'diamond'},
    connectgaps=True,
    line=dict(color='#495057')
))

fig.add_trace(go.Scatter(
    x=laws,
    y=results_c,
    mode='markers',
    name='without borrowing',
    marker={'size':msize},
    connectgaps=True,
    line=dict(color='#343A40')
))

fig.add_trace(go.Scatter(
    x=laws,
    y=results_e,
    mode='markers',
    name='with constant interest rates',
    marker={'size':msize+2,'symbol':'triangle-up'},
    connectgaps=True,
    line=dict(color='#8D959D')
))

# fig.add_trace(go.Scatter(
#     x=laws,
#     y=results_d,
#     mode='markers',
#     name='with revaluation effect',
#     marker={'size':msize+2,'symbol':'triangle-up'},
#     connectgaps=True,
#     line=dict(color='green')
# ))

fig.update_layout(
    yaxis=dict(
    tickformat='.1%'
    ),
    width=700,
    legend=dict(
        yanchor="bottom",
        title="",
        y=1,
        xanchor="center",
        x=0.5,
        orientation="h",
        font=dict(size=15,family="serif")
    ))

fig.write_image('fig4.pdf')
fig.show()

With 10-2022 expectations, real PVS are much lower than with 2020-expectations
This is primarily driven by the discount factor, which is much lower in 2022
because the nominal interest rate expectations are higher

See comparison of 10-2022 vs 1-2020:

In [ ]:
comparison = price_level_estimation(output='dataframe',fiscal_balances = df2022['Primary Surplus Total'],
        discount_factors = discount_factors("10-2022")) / price_level_estimation(output='dataframe',fiscal_balances = df2022['Primary Surplus Total'], 
        discount_factors = discount_factors("1-2020"))
comparison.tail()

## Sensitvity Checks

In [ ]:
standard_discount = discount_factors("10-2022")
df2022 = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="10-2022",
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])
p2022_baseline = price_level_estimation(
        fiscal_balances = df2022['Primary Surplus Total'],
        discount_factors = standard_discount)

# Interest rate changes (+/i 1 pp)

df_interest_lower = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="10-2022",
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0],
                                    deviation_interest_rate=-1)
p2022_interest_lower = price_level_estimation(
        fiscal_balances = df_interest_lower['Primary Surplus Total'],
        discount_factors = standard_discount)

df_interest_upper = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="10-2022",
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0],
                                    deviation_interest_rate=+1)
p2022_interest_upper  = price_level_estimation(
        fiscal_balances = df_interest_upper['Primary Surplus Total'], 
        discount_factors = standard_discount)

# GDP changes (+/- 1pp growth)

df_gdp_lower = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="10-2022",
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0],
                                    growth_deviation=-0.01)
p2022_gdp_lower = price_level_estimation(
        fiscal_balances = df_gdp_lower['Primary Surplus Total'],
        discount_factors = standard_discount)

df_gdp_upper = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="10-2022",
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0],
                                    growth_deviation=0.01)
p2022_gdp_upper = price_level_estimation(
        fiscal_balances = df_gdp_upper['Primary Surplus Total'],
        discount_factors = standard_discount)

# Discount changes (+/- 1 pp)

p2022_discount_lower= price_level_estimation(
        fiscal_balances = df2022['Primary Surplus Total'],
        discount_factors = discount_factors("10-2022",deviation=1))

p2022_discount_upper = price_level_estimation(
        fiscal_balances = df2022['Primary Surplus Total'],
        discount_factors = discount_factors("10-2022",deviation=-1))

In [ ]:
scenario=['<i>GDP growth</i>', '<i>Discount rates</i>', '<i>Interest rates</i>']
upper = [p2022_gdp_upper/p2020_0-1,
          p2022_discount_upper/p2020_0-1,
          p2022_interest_upper/p2020_0-1]
lower = [p2022_gdp_lower/p2020_0-1,
          p2022_discount_lower/p2020_0-1,
          p2022_interest_lower/p2020_0-1]
baseline = [p2022_baseline/p2020_0-1, p2022_baseline/p2020_0-1, p2022_baseline/p2020_0-1, p2022_baseline/p2020_0-1]

## Figure 5
<a id='fig5'></a>

In [ ]:
fig = go.Figure()
fig.update_layout(template="simple_white",
                 xaxis=dict(showgrid=False,tickfont = dict(size=16,family="serif"),
                            title="Factor changed by +/- 1 percentage point",titlefont = dict(size=19,family="serif")), 
                 yaxis=dict(showgrid=True,tickfont = dict(size=16,family="serif"),tickformat="0%",
                            title="PLE relative to 1-2020",titlefont = dict(size=19,family="serif")))
msize=12

fig.add_trace(go.Scatter(
    x=scenario,
    y=upper,
    mode='markers',
    name='Upper range',
    marker={'size':msize,'color':'black','symbol':'triangle-up'},
    connectgaps=True)
)

fig.add_trace(go.Scatter(
    x=scenario,
    y=lower,
    mode='markers',
    name='Lower range',
    marker={'size':msize,'color':'black','symbol':'triangle-down'},
    connectgaps=True)
)

fig.add_trace(go.Scatter(
    x=scenario,
    y=baseline,
    mode='markers',
    name='Baseline',
    marker={'size':msize-3,'color':'black','symbol':'square'},
    connectgaps=True)
)

fig.update_layout(
    yaxis=dict(
    tickformat='.1%'
    ),
    width=700,
    legend=dict(
        yanchor="bottom",
        title="",
        y=1,
        xanchor="center",
        x=0.5,
        orientation="h",
        font=dict(size=15,family="serif")
    ))

fig.write_image('fig5.pdf')
fig.show()

### Schuldenbremse Reform

Idea: Allow 1% of GDP borrowing for total government (0.7% for federal and 0.3% for states)

Results: Does not matter much

In [ ]:
df2022 = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="10-2022",
                                    rule_states=0.003,rule_bund=0.007,
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])
p2022a_r = price_level_estimation(
        fiscal_balances = df2022['Primary Surplus Total'],
        discount_factors = discount_factors("10-2022"))
    # ex post 2021
p2022c_r = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="10-2022",
                                    rule_states=0.003,rule_bund=0.007,
                                    emergency_credit_bund = [0,0,0],
                                    emergency_credit_states = [0,0,0])['Primary Surplus Total'], 
        discount_factors = discount_factors("10-2022"))
    # ex post 2021 (no borrowing)
p2022e_r = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2020",
                                    rule_states=0.003,rule_bund=0.007,
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])['Primary Surplus Total'], 
        discount_factors = discount_factors("10-2022"))
    # ex post 2021 (constant interest rates)

df2022_2 = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2022",
                                    rule_states=0.003,rule_bund=0.007,
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])
p2022_2a_r = price_level_estimation(
        fiscal_balances = df2022_2['Primary Surplus Total'], 
        discount_factors = discount_factors("1-2022"))
    # ex post at 1-2021
p2022_2c_r = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2022",
                                    rule_states=0.003,rule_bund=0.007,
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("1-2022"))
    # ex post at 1-2021 (constant interest rates)
p2022_2e_r = price_level_estimation(
        fiscal_balances = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="1-2020",
                                    rule_states=0.003,rule_bund=0.007,
                                    emergency_credit_bund = [0,0,0],
                                    emergency_credit_states = [0,0,0])['Primary Surplus Total'],
        discount_factors = discount_factors("1-2022"))
    # ex post at 1-2021 (no borrowing)

## Figure 6a
<a id='fig6a'></a>

In [ ]:
laws=['Ex post (1-2022)', '<i>Reform (1-2022)</i>', 'Ex post (10-2022)', '<i>Reform (10-2022)</i>']
results_a_r = [p2022_2a/p2020_0-1, p2022_2a_r/p2020_0-1, p2022a/p2020_0-1, p2022a_r/p2020_0-1]
results_c_r = [p2022_2c/p2020_0-1, p2022_2c_r/p2020_0-1, p2022c/p2020_0-1, p2022c_r/p2020_0-1]
results_e_r = [p2022_2e/p2020_0-1, p2022_2e_r/p2020_0-1, p2022e/p2020_0-1, p2022e_r/p2020_0-1]

In [ ]:
fig = go.Figure()
fig.update_layout(template="simple_white",
                 xaxis=dict(showgrid=False,tickfont = dict(size=17,family="serif"),
                            title="Federal Budget Laws (Date)",titlefont = dict(size=19,family="serif")), 
                 yaxis=dict(showgrid=True,tickfont = dict(size=17,family="serif"),tickformat="0%",
                            title="PLE relative to 1-2020",titlefont = dict(size=19,family="serif")))
msize=8
fig.add_trace(go.Scatter(
    x=laws,
    y=results_a_r,
    mode='markers',
    name='Price Level Effect (PLE)',
    marker={'size':msize},
    connectgaps=True,
    line=dict(color='#343A40')
))

fig.add_trace(go.Scatter(
    x=laws,
    y=results_c_r,
    mode='markers',
    name='without borrowing',
    marker={'size':msize},
    connectgaps=True,
    line=dict(color='#CED4DA')
))


fig.add_trace(go.Scatter(
    x=laws,
    y=results_e_r,
    mode='markers',
    name='with constant interest rates',
    marker={'size':msize},
    connectgaps=True,
    line=dict(color='#8D959D')
))

fig.update_layout(
    yaxis=dict(
    tickformat='.1%'
    ),
    width=700,
    legend=dict(
        yanchor="bottom",
        title="",
        y=1,
        xanchor="center",
        x=0.5,
        orientation="h",
        font=dict(size=15,family="serif")
    ))

fig.update_layout(
    margin=dict(l=0,r=0,b=0,t=100)
    )

fig.write_image('fig6a.pdf')
fig.show()

## Figure 7
<a id='fig7'></a>

Historical values from Ameco

In [ ]:
# For years 2015-2019(2024): 
gdp = [3026,3132,3267,3365,3474,3405,3602]#,3867,4112,4268]
interest = [42.2,37.3,33.8,31.2,27.4,21.5,20.8]#,26.1,33.3,38.4]
relative_interest = []
for i in range(0,6):
    relative_interest.append(interest[i]/gdp[i])

In [ ]:
interest_grimm = pd.read_excel("data.xlsx", sheet_name='Grimm2022')
interest_grimm = interest_grimm.iloc[10:]

In [ ]:
fig = go.Figure()
fig.update_layout(template="simple_white",
                 xaxis=dict(showgrid=False,tickfont = dict(size=17,family="serif"),range=[2015,2040],
                            title="",titlefont = dict(size=19,family="serif")), 
                 yaxis=dict(showgrid=True,tickfont = dict(size=17,family="serif"),tickformat="0%",range=[0,0.02],
                            title="Interest in % of GDP",titlefont = dict(size=19,family="serif")))

fig.add_trace(go.Line(x=list(range(2015,2022)),y=relative_interest,name="Historic",line=dict(color='black'),mode="lines"))
fig.add_trace(go.Line(x=list(range(2020,2051)),y=df2021_2['Interest Bund']/df2021_2['GDP'],name='S I 2021 (6-2022)',line=dict(color='#6C757D')))
fig.add_trace(go.Line(x=list(range(2020,2051)),y=df2022['Interest Bund']/df2022['GDP'],name='Ex post (10-2022)',line=dict(color='#343A40')))
fig.add_trace(go.Line(x=list(range(2020,2035)),y=interest_grimm['YC 2022']/100,name="<i>YC 2022</i>",line=dict(color='#6C757D',dash='dash'),mode="lines"))
fig.add_trace(go.Line(x=list(range(2020,2035)),y=interest_grimm['YC 2022 + 1pp']/100,name='<i>YC 2022 + 1pp</i>',line=dict(color='#343A40',dash='dash'),mode="lines"))
fig.add_trace(go.Line(x=list(range(2020,2035)),y=interest_grimm['YC 2022 + 2pp']/100,name='<i>YC 2022 + 2pp</i>',line=dict(color='#131517',dash='dash'),mode="lines"))

fig.update_layout(
    yaxis=dict(
    tickformat='.1%'
    ),
    width=700,
    legend=dict(
        yanchor="bottom",
        title="",
        y=1,
        xanchor="center",
        x=0.5,
        orientation="h",
        font=dict(size=18,family="serif")
    ))

fig.write_image('fig7.pdf')
fig.show()

In [ ]:
# For years 2015-2019:
gdp = [3026,3132,3267,3365,3474]
federal_budget = [312,317,331,348,357]
budget_size = []
for i in range(0,5):
    budget_size.append(federal_budget[i]/gdp[i])
# Budget size historically ~10% of GDP

## Figure 2b
<a id='fig2b'></a>

In [ ]:
# Assuming a federal budget size of 10.% of nominal GDP
df2020_0 = nominal_fiscal_balances(output='dataframe', baseyear=2020)

relative_interest2 = []
for i in range(0,5):
    relative_interest2.append(interest[i]/federal_budget[i])
relative_interest2

fig = go.Figure()
fig.update_layout(template="simple_white",
                 xaxis=dict(showgrid=False,tickfont = dict(size=17,family="serif"),
                            title="",titlefont = dict(size=18,family="serif")), 
                 yaxis=dict(showgrid=True,tickfont = dict(size=17,family="serif"),tickformat="0%",
                            title="Interest expenditures in % of federal budget",titlefont = dict(size=18,family="serif")))

fig.add_trace(go.Line(x=list(range(2015,2020)),y=relative_interest2,name="Historic",line=dict(color='black'),mode="lines"))
fig.add_trace(go.Line(x=list(range(2020,2051)),y=df2020_0['Interest Bund']/df2020_0['GDP']/0.105,name="Baseline",line=dict(color='black',dash='dash')))
fig.add_trace(go.Line(x=list(range(2020,2051)),y=df2020_1['Interest Bund']/df2020_1['GDP']/0.105,name='S I 2020',line=dict(color='#495057')))
fig.add_trace(go.Line(x=list(range(2020,2051)),y=df2020_2['Interest Bund']/df2020_2['GDP']/0.105,name='S II 2020',line=dict(color='#064046')))
fig.add_trace(go.Line(x=list(range(2020,2051)),y=df2021_2['Interest Bund']/df2021_2['GDP']/0.105,name='S I 2021',line=dict(color='#ADB5BD')))
fig.add_trace(go.Line(x=list(range(2020,2051)),y=df2022['Interest Bund']/df2022['GDP']/0.105,name='Ex post (10-2022)',line=dict(color='#343A40')))

fig.update_layout(
    showlegend=True,
    yaxis=dict(
    tickformat='.1%'
    ),
    width=700,
    legend=dict(
        yanchor="bottom",
        title="",
        y=1.1,
        xanchor="center",
        x=0.5,
        orientation="h",
        font=dict(size=18,family="serif")
    ))

fig.write_image("fig2b.pdf")
fig.show()

## Constructed Time-Series

## Figure 1
<a id='fig1'></a>

In [ ]:
df_fig2 = nominal_fiscal_balances(output='dataframe', baseyear=2020, ex_post=1,
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])
df_fig2['Bund'] = - df_fig2['NKA Bund'] / df_fig2['GDP']
df_fig2['States'] = - df_fig2['NKA States'] / df_fig2['GDP']
df_fig2['General Government'] = - (df_fig2['NKA States']+df_fig2['NKA Bund'])/ df_fig2['GDP']
fig = px.bar(df_fig2, x='Year',y=['States','Bund'],
              color_discrete_sequence=["#E3E6E8","#ADB5BD","#6C757D","#343A40","#131517"])
fig.update_layout(template="simple_white",
                 xaxis=dict(showgrid=False,tickfont = dict(size=17,family="serif"),title=""), 
                 yaxis=dict(range=[-0.095,0.005],showgrid=True,tickfont = dict(size=17,family="serif"),tickformat="0%",
                            title="Net lending/borrowing in % of GDP",titlefont = dict(size=19,family="serif")))

fig.add_trace(go.Scatter(
    x=df_fig2['Year'],
    y=df_fig2['General Government'],
    mode='lines',
    name='General Government',
    connectgaps=True,
    line=dict(color='#343A40')
))
    
fig.update_layout(
    yaxis=dict(
    tickformat='.1%'
    ),
    width=700,
    legend=dict(
        yanchor="bottom",
        title="",
        y=-0.23,
        xanchor="center",
        x=0.5,
        orientation="h",
        font=dict(size=19,family="serif")
    ))

fig.update_layout(
    margin=dict(l=0,r=0,b=0,t=50)
    )

fig.write_image("fig1.pdf")
fig.show()

In [ ]:
df2022 = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="10-2022",
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])
df2022_reform = nominal_fiscal_balances(output='dataframe', baseyear=2021, date="10-2022",
                                    rule_states=0.003,rule_bund=0.007,
                                    emergency_credit_bund = [41.9,192,0],
                                    emergency_credit_states = [30,0,0])

## Figure 6b
<a id='fig6b'></a>

In [ ]:
fig = go.Figure()
fig.update_layout(template="simple_white",
                 xaxis=dict(showgrid=False,tickfont = dict(size=17,family="serif"),title=""), 
                 yaxis=dict(range=[0.05,0.75],showgrid=True,tickfont = dict(size=17,family="serif"),tickformat="0%",
                            title="Government debt in % of GDP",titlefont = dict(size=19,family="serif")))

fig.add_trace(go.Scatter(
    x=df2022['Year'],
    y=df2022['Debt Total']/df2022['GDP'],
    mode='lines',
    name='Baseline',
    connectgaps=True,
    line=dict(color='#131517')
))

fig.add_trace(go.Scatter(
    x=df2022['Year'],
    y=df2022_reform['Debt Total']/df2022['GDP'],
    mode='lines',
    name='Debt rule reform',
    connectgaps=True,
    line=dict(color='#ADB5BD')
))
    
fig.update_layout(
    yaxis=dict(
    tickformat='.1%'
    ),
    width=700,
    legend=dict(
        yanchor="bottom",
        title="",
        y=-0.23,
        xanchor="center",
        x=0.5,
        orientation="h",
        font=dict(size=19,family="serif")
    ))

fig.update_layout(
    margin=dict(l=0,r=0,b=0,t=50)
    )


fig.write_image("fig6b.pdf")
fig.show()

### Figure 3

In [ ]:
def datensammlung_import(sheet):
    df = pd.read_excel("data.xlsx", sheet_name=sheet).set_index("Unnamed: 0")
    dates = []
    for y in range(2018,2054):
        for m in range(1,13):
            if len(dates) < (len(df.columns)+30*12):
                dates.append(str(m)+"-"+str(y))
            else:
                break
    output = pd.DataFrame(dates)

    count = 0
    countdown = 66
    for y in range(2018,2024):
        for m in range(1,13):
            if (m==7) & (y==2023):
                break
            else:
                #print(str(m)+"-"+str(y))
                series = [None]*count
                series += [(item / 100) if item is not None else None for sublist in [i for i in [[i]+[None]*11 for i in list(df[str(m)+"-"+str(y)])]] for item in sublist]
                series += [None]*countdown
                output[str(m)+"-"+str(y)] = series
                count += 1
                countdown -=1
    output = output.rename(columns={0:'Future Date'})
    output['Future Time'] = pd.to_datetime(output['Future Date'])
    return output

def datensammlung_figure(data,title="",filename="graph",note="",ytitle=""):
    fig = go.Figure()
    fig.update_layout(template="simple_white",
                     xaxis=dict(showgrid=False,tickfont = dict(size=16,family="serif")), 
                     yaxis=dict(showgrid=True,tickfont = dict(size=16,family="serif"),
                               title=ytitle,titlefont=dict(size=19,family="serif")))

    #colors = ["navy", "green", "#81AB20", "#FF5300", "red"]
    colors = ["#495057", "#ADB5BD", "#CED4DA", "#6C757D", "#343A40"]
    dash = ["dashdot", "dot", "dot" , "dash", "solid"]
    lines = ["1-2020","1-2021","5-2021","1-2022","10-2022"]
    names = ["1-2020","1-2021","5-2021","1-2022","10-2022"]
    #markers = ["square", "triangle-up", "triangle-down", "circle"]
    
    for i,line in enumerate(lines):
        fig.add_trace(go.Scatter(
            x=data['Future Time'],
            y=data[line],
            mode='lines',
            name=names[i % len(names)],
            connectgaps=True,
            line=dict(color=colors[i % len(colors)],dash=dash[i % len(dash)],width=3.5),
            #marker=dict(symbol=markers[i % len(markers)],size=9.5)
        ))

    fig.update_layout(
        #title=f"<b>{title}</b>",
        #width=750,
        yaxis=dict(
        tickformat='.1%'
        ),
        #margin=dict(t=0,b=0,l=0,r=0),
        legend=dict(
            yanchor="bottom",
            y=-0.23,
            xanchor="center",
            x=0.5,
            orientation="h",
            #bordercolor="black",
            #borderwidth=1,
            font=dict(size=16,family="serif")
        ))
    
    fig.write_image(filename+".pdf",height=600,width=900)
    fig.show()

## Figure 3
<a id='fig3'></a>

In [ ]:
datensammlung_figure(datensammlung_import("EA E(i) from YC"), "Expected short-term nominal interest rate",filename="fig3",
                    note=f"Derived from AAA-Yield Curve by ECB", ytitle="Expected short-term nominal interest rate")